# Testing Our Models #

We will test the stability of the solutions returned by each model by running 100 randomly generated tests for each model type.

In [1]:
from biglittlematcher import BigLittleMatcher
import random

This will be our general function for testing stability.

In [2]:
def test_stability(generator_func, build_model=None, num_tests=100):
    stable_count = 0
    unstable_count = 0
    
    for _ in range(num_tests):
        if build_model:
            bigs, littles, big_prefs, little_prefs, model_builder = generator_func(build_model)
        else:
            bigs, littles, big_prefs, little_prefs, model_builder = generator_func()
        
        try:
            matcher = BigLittleMatcher(bigs, littles, big_prefs, little_prefs)
            model_builder(matcher)
            matches, _ = matcher.solve()
            instabilities = matcher.check_instabilities(matches)
            
            if len(instabilities) == 0:
                stable_count += 1
            else:
                unstable_count += 1
        except Exception as e:
            unstable_count += 1
    
    return stable_count, unstable_count

First, we will test all the models with the standard stable marriage problem.

In [3]:
def generate_sm_test(build_model_func="build_model"):
    size = random.randint(3, 6)
    bigs = {f"B{i}": {} for i in range(1, size+1)}
    littles = {f"L{i}": {} for i in range(1, size+1)}
    
    big_prefs = {}
    little_prefs = {}
    
    # Choose format based on which build model we're using
    for b in bigs:
        if build_model_func == "build_model":
            big_prefs[b] = random.sample(list(littles.keys()), len(littles))
        else:
            littles_list = random.sample(list(littles.keys()), len(littles))
            big_prefs[b] = {littles_list[i]: i+1 for i in range(len(littles_list))}

    for l in littles:
        if build_model_func == "build_model":
            little_prefs[l] = random.sample(list(bigs.keys()), len(bigs))
        else:
            bigs_list = random.sample(list(bigs.keys()), len(bigs))
            little_prefs[l] = {bigs_list[i]: i+1 for i in range(len(bigs_list))}
    
    model_builder = None
    if build_model_func == "build_model":
        model_builder = lambda m: m.build_model()
    elif build_model_func == "build_model_smt":
        model_builder = lambda m: m.build_model_smt()
    elif build_model_func == "build_model_smti":
        model_builder = lambda m: m.build_model_smti()
    
    return bigs, littles, big_prefs, little_prefs, model_builder

### Standard Matching (Original) Test Results ###

The results here look good because all models should be able to solve the standard matching problem. Since SMT and SMTI are both supersets of the original model.

In [4]:
# Run 100 tests for each model type
print("Testing Standard Stable Matching with build_model:")
sm_stable, sm_unstable = test_stability(generate_sm_test, "build_model")
print(f"Stable: {sm_stable}, Unstable: {sm_unstable}")

print("Testing Standard Stable Matching with build_model_smt:")
sm_stable2, sm_unstable2 = test_stability(generate_sm_test, "build_model_smt")
print(f"Stable: {sm_stable2}, Unstable: {sm_unstable2}")

print("Testing Standard Stable Matching with build_model_smti:")
sm_stable2, sm_unstable2 = test_stability(generate_sm_test, "build_model_smti")
print(f"Stable: {sm_stable2}, Unstable: {sm_unstable2}")

Testing Standard Stable Matching with build_model:
Stable: 100, Unstable: 0
Testing Standard Stable Matching with build_model_smt:
Stable: 100, Unstable: 0
Testing Standard Stable Matching with build_model_smti:
Stable: 100, Unstable: 0


Then, we will test the models with the stable matching with ties problem.

In [5]:
def generate_smt_test(build_model_func="build_model_smt"):
    size = random.randint(3, 6)
    bigs = {f"B{i}": {} for i in range(1, size+1)}
    littles = {f"L{i}": {} for i in range(1, size+1)}
    
    # Generate preferences with ties
    big_prefs = {}
    little_prefs = {}
    
    for b in bigs:
        # Complete list but with possible ties
        ranks = {}
        little_list = list(littles.keys())
        random.shuffle(little_list)
        
        current_rank = 1
        for l in little_list:
            # 30% chance of keeping the same rank (making a tie)
            if random.random() > 0.3 and l != little_list[0]:
                current_rank += 1
            ranks[l] = current_rank
        big_prefs[b] = ranks
    
    for l in littles:
        ranks = {}
        big_list = list(bigs.keys())
        random.shuffle(big_list)
        
        current_rank = 1
        for b in big_list:
            # 30% chance of keeping the same rank (making a tie)
            if random.random() > 0.3 and b != big_list[0]:
                current_rank += 1
            ranks[b] = current_rank
        little_prefs[l] = ranks
    
    # Map string name to actual method call
    model_builder = None
    if build_model_func == "build_model_smt":
        model_builder = lambda m: m.build_model_smt()
    elif build_model_func == "build_model_smti":
        model_builder = lambda m: m.build_model_smti()
    
    return bigs, littles, big_prefs, little_prefs, model_builder

### Stable Matching with Ties Test Results ###

The results here look good because SMT and SMTI both solve the stable matching with ties problem. This additionally looks good because SMTI is a superset of SMT.

In [6]:

print("\nTesting Stable Matching with Ties using build_model_smt:")
smt_stable, smt_unstable = test_stability(generate_smt_test, "build_model_smt")
print(f"Stable: {smt_stable}, Unstable: {smt_unstable}")

print("\nTesting Stable Matching with Ties using build_model_smti:")
smt_stable2, smt_unstable2 = test_stability(generate_smt_test, "build_model_smti")
print(f"Stable: {smt_stable2}, Unstable: {smt_unstable2}")


Testing Stable Matching with Ties using build_model_smt:
Stable: 100, Unstable: 0

Testing Stable Matching with Ties using build_model_smti:
Stable: 100, Unstable: 0


Finally, we will test the models with the stable matching with ties and incomplete lists problem.

In [7]:
def generate_smti_test():
    """Generate a Stable Marriage with Ties and Incomplete Lists test case"""
    size = random.randint(3, 6)
    bigs = {f"B{i}": {} for i in range(1, size+1)}
    littles = {f"L{i}": {} for i in range(1, size+1)}
    
    # Generate preferences with ties and incomplete lists
    big_prefs = {}
    little_prefs = {}
    
    for b in bigs:
        # Each big ranks a random subset of littles
        num_to_rank = random.randint(1, len(littles))
        selected_littles = random.sample(list(littles.keys()), num_to_rank)
        
        ranks = {}
        current_rank = 1
        for l in selected_littles:
            # 30% chance of keeping the same rank (making a tie)
            if random.random() > 0.3 and l != selected_littles[0]:
                current_rank += 1
            ranks[l] = current_rank
        big_prefs[b] = ranks
    
    for l in littles:
        # Each little ranks a random subset of bigs
        num_to_rank = random.randint(1, len(bigs))
        selected_bigs = random.sample(list(bigs.keys()), num_to_rank)
        
        ranks = {}
        current_rank = 1
        for b in selected_bigs:
            # 30% chance of keeping the same rank (making a tie)
            if random.random() > 0.3 and b != selected_bigs[0]:
                current_rank += 1
            ranks[b] = current_rank
        little_prefs[l] = ranks
    
    return bigs, littles, big_prefs, little_prefs, lambda m: m.build_model_smti()

### Stable Matching with Ties and Incomplete Lists Test Results ###

The results here look good because SMTI solver solves the stable matching with ties and incomplete lists problem.

In [8]:
print("\nTesting Stable Matching with Ties and Incomplete Lists:")
smti_stable, smti_unstable = test_stability(generate_smti_test)
print(f"Stable: {smti_stable}, Unstable: {smti_unstable}")


Testing Stable Matching with Ties and Incomplete Lists:
Stable: 100, Unstable: 0
